# Data Import

## Google Drive

# Resnet

## Import Modules

In [1]:
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt


from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def implt(img):
    plt.figure()
    plt.imshow(img)
    plt.axis('off')

                            # Set up 'ggplot' style
plt.style.use('ggplot')     # if want to use the default style, set 'classic'
plt.rcParams['ytick.right']     = True
plt.rcParams['ytick.labelright']= True
plt.rcParams['ytick.left']      = False
plt.rcParams['ytick.labelleft'] = False
plt.rcParams['font.family']     = 'Arial'

project_dir = r"D:\NUS_TERM2_CA2"
train_dir = project_dir + "\Train"
validation_dir = project_dir + "\Validation"


seed        = 29
np.random.seed(seed)

optmz       = optimizers.Adam(lr=0.001)
modelname   = 'ca2_jiahao'

## Layer Define

In [2]:
# resLyr
def resLyr(inputs,
           numFilters=16,
           kernelSz=3,
           strides=1,
           activation='relu',
           batchNorm=True,
           convFirst=True,
           lyrName=None):
    convLyr = Conv2D(numFilters,
                       kernel_size=kernelSz,
                       strides=strides,
                       padding='same',
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4),
                       name=lyrName+'_conv' if lyrName else None)
    x = inputs
    if convFirst:
        x = convLyr(x)
        if batchNorm:
            x = BatchNormalization(name=lyrName+'_bn' if lyrName else None)(x)
        if activation is not None:
            x = Activation(activation,name=lyrName+'_'+activation if lyrName else None)(x)
    else:
        if batchNorm:
            x = BatchNormalization(name=lyrName+'_bn' if lyrName else None)(x)
        if activation is not None:
            x = Activation(activation,name=lyrName+'_'+activation if lyrName else None)(x)
        x = convLyr(x)

    return x

# resBlkV1
def resBlkV1(inputs,
             numFilters=16,
             numBlocks=3,
             downsampleOnFirst=True,
             names=None):
    x = inputs
    
    for run in range(0,numBlocks):
        strides = 1
        blkStr = str(run+1)

        if downsampleOnFirst and run == 0:
            strides = 2

        y = resLyr(inputs=x,
                   numFilters=numFilters,
                   strides=strides,
                   lyrName=names+'_Blk'+blkStr+'_Res1' if names else None)
        y = resLyr(inputs=y,
                   numFilters=numFilters,
                   activation=None,
                   lyrName=names+'_Blk'+blkStr+'_Res2' if names else None)
        
        if downsampleOnFirst and run == 0:
            x = resLyr(inputs=x,
                        numFilters=numFilters,
                        kernelSz=1,
                        strides=strides,
                        activation=None,
                        batchNorm=False,
                        lyrName=names+'_Blk'+blkStr+'_lin' if names else None)
            
        x = add([x,y],name=names+'_Blk'+blkStr+'_add' if names else None)
        x = Activation('relu',name=names+'_Blk'+blkStr+'_relu' if names else None)(x)

    return x

# createResNetV1
def createResNetV1(inputShape=(32,32,3),
                   numClasses=3):
    inputs = Input(shape=inputShape)
    v = resLyr(inputs, lyrName='Inpt')
    v = resBlkV1(v,16,3,False,'Stg1')
    v = resBlkV1(v,32,3,True,'Stg2')
    v = resBlkV1(v,64,3,True,'Stg3')
    v = resBlkV1(v,128,3,True,'Stg4')
    v = AveragePooling2D(pool_size=4,name='AvgPool')(v)
    v = Flatten()(v)
    outputs = Dense(numClasses,activation='softmax',kernel_initializer='he_normal')(v)
    model = Model(inputs=inputs,outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                    optimizer=optmz,
                    metrics=['accuracy'])

    return model

# set up model
model       = createResNetV1()  # This is meant for training
modelGo     = createResNetV1()  # This is used for final testing

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Inpt_conv (Conv2D)              (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
Inpt_bn (BatchNormalization)    (None, 32, 32, 16)   64          Inpt_conv[0][0]                  
__________________________________________________________________________________________________
Inpt_relu (Activation)          (None, 32, 32, 16)   0           Inpt_bn[0][0]                    
______________________________________________________________________________________________

## Learning Rate Define

In [3]:
def lrSchedule(epoch):
    lr  = 1e-3
    
    if epoch > 160:
        lr  *= 0.5e-3        
    elif epoch > 140:
        lr  *= 1e-3       
    elif epoch > 120:
        lr  *= 1e-2
    elif epoch > 80:
        lr  *= 1e-1
        
    print('Learning rate: ', lr)
    
    return lr

LRScheduler     = LearningRateScheduler(lrSchedule)

## Checkpoint

In [4]:
# Create checkpoint for the training
# This checkpoint performs model saving when
# an epoch gives highest testing accuracy
filepath        = project_dir + "\\" + modelname + ".hdf5"
checkpoint      = ModelCheckpoint(filepath, 
                                  monitor='val_acc', 
                                  verbose=0, 
                                  save_best_only=True, 
                                  mode='max')

# Log the epoch detail into csv
csv_logger      = CSVLogger(project_dir + "\\" + modelname +'.csv')

# Tensorboard
# tensorboard     = Tensorboard(log_dir=project_dir)
callbacks_list  = [checkpoint,csv_logger,LRScheduler]

## Fit Model

In [6]:
train_datagen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             rotation_range=20,
                             horizontal_flip=True,
                             vertical_flip=False)

test_datagen = ImageDataGenerator()

num_train_images=2423

batch_size = 32

model.fit_generator(train_datagen.flow_from_directory(train_dir,
                                                      target_size=(32, 32), 
                                                      batch_size=batch_size),
                    validation_data=test_datagen.flow_from_directory(validation_dir,
                                                                     target_size=(32, 32), 
                                                                     batch_size=batch_size),
                    epochs=40, 
                    workers=12,
                    verbose=1,
                    steps_per_epoch=num_train_images//batch_size,
                    # validation_steps=//batch_size,
                    callbacks=callbacks_list)

Found 2423 images belonging to 3 classes.
Found 724 images belonging to 3 classes.
Learning rate:  0.001
Epoch 1/40
75/75 [==============================] - 43s 574ms/step - loss: 0.8143 - acc: 0.8168 - val_loss: 1.3941 - val_acc: 0.6036
Learning rate:  0.001
Epoch 2/40
75/75 [==============================] - 43s 574ms/step - loss: 0.8238 - acc: 0.8087 - val_loss: 0.8472 - val_acc: 0.7804
Learning rate:  0.001
Epoch 3/40
75/75 [==============================] - 42s 560ms/step - loss: 0.7764 - acc: 0.8287 - val_loss: 0.8322 - val_acc: 0.7887
Learning rate:  0.001
Epoch 4/40
75/75 [==============================] - 42s 557ms/step - loss: 0.7173 - acc: 0.8453 - val_loss: 0.7729 - val_acc: 0.8425
Learning rate:  0.001
Epoch 5/40
75/75 [==============================] - 41s 548ms/step - loss: 0.7243 - acc: 0.8386 - val_loss: 0.7879 - val_acc: 0.8177
Learning rate:  0.001
Epoch 6/40
75/75 [==============================] - 42s 564ms/step - loss: 0.6987 - acc: 0.8457 - val_loss: 1.1155 - val

## Evaluation

In [0]:
# ......................................................................


                            # Now the training is complete, we get
                            # another object to load the weights
                            # compile it, so that we can do 
                            # final evaluation on it
modelGo.load_weights(filepath)
modelGo.compile(loss='categorical_crossentropy', 
                optimizer=optmz, 
                metrics=['accuracy'])

# .......................................................................


                            # Make classification on the test dataset
predicts    = modelGo.predict(tsDat)


                            # Prepare the classification output
                            # for the classification report
predout     = np.argmax(predicts,axis=1)
testout     = np.argmax(tsLbl,axis=1)
labelname   = ["food", "landmark", "people"]
                                            # the labels for the classfication report


testScores  = metrics.accuracy_score(testout,predout)
confusion   = metrics.confusion_matrix(testout,predout)


print("Best accuracy (on testing dataset): %.2f%%" % (testScores*100))
print(metrics.classification_report(testout,predout,target_names=labelname,digits=4))
print(confusion)


    
    
    
# ..................................................................
    
import pandas as pd

records     = pd.read_csv(project_dir + "/" + modelname +'.csv')
plt.figure()
plt.subplot(211)
plt.plot(records['val_loss'])
plt.plot(records['loss'])
plt.yticks([0,0.20,0.40,0.60,0.80,1.00])
plt.title('Loss value',fontsize=12)

ax          = plt.gca()
ax.set_xticklabels([])



plt.subplot(212)
plt.plot(records['val_acc'])
plt.plot(records['acc'])
plt.yticks([0.6,0.7,0.8,0.9,1.0])
plt.title('Accuracy',fontsize=12)
plt.show()




from tensorflow.keras.utils import plot_model

plot_model(model, 
           to_file=project_dir + "/" + modelname+'_model.pdf', 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='TB')